In [ ]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-4o-mini")
chat = ChatOpenAI(model_name="gpt-4o-mini")

a = llm.predict("안녕 넌 한글도 알고 있니?")
b = chat.predict("How may planets are there ?")

a, b

# ChatOpenAI: OpenAI의 챗 모델과 상호작용하는 객체로, predict나 predict_messages 메서드를 통해 완성된 프롬프트를 모델에 전달하고 응답을 받는다.
# OpenAI: OpenAI의 LLM 모델과 상호작용하는 객체로, predict 메서드를 통해 완성된 프롬프트를 모델에 전달하고 응답을 받는다.

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1);

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand? Also, What is your name?"),
]

chat.predict_messages(messages)

# 이 코드는 LangChain 라이브러리를 사용하여 OpenAI의 Chat 모델 인스턴스를 생성한 뒤, 시스템 메시지(역할 지시), AI 응답 예시, 그리고 사용자 메시지를 순서대로 담은 리스트를 구성한다.
# 이후 predict_messages 메서드를 호출하여 주어진 문맥(메시지 리스트)에 따른 모델의 새로운 응답을 얻는 간단한 예시 코드이다.
# 	1.	ChatOpenAI 클래스를 통해 gpt-4o-mini 모델과 temperature=0.1 설정을 가진 챗 모델 인스턴스를 생성한다.
#   (temperature는 모델이 응답을 생성할 때 정답으로부터 얼마나 더 창의적이고 다양하게 벗어날 수 있는지를 제어하는 파라미터로, 숫자가 높을수록 더 다양한 결과를, 낮을수록 더 일관되고 예측 가능한 결과를 낸다.)
# 	2.	시스템 메시지를 통해 모델에게 지리학 전문가 역할을 부여하고, 이탈리아어로만 답변하도록 지시한다.
# 	3.	이전 대화 흐름(예: “Ciao, mi chiamo Paolo!”)와 함께, 사용자의 질문(멕시코와 태국 사이의 거리와 이름 문의)이 담긴 메시지 목록을 만든다.
# 	4.	chat.predict_messages(messages)를 통해 주어진 대화 흐름에 대한 모델의 답변을 얻는다.


In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate;

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1);

simpleTemplate = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
simplePrompt = simpleTemplate.format(country_a="Mexico", country_b="Thailand");
chat.predict(simplePrompt)

template = ChatPromptTemplate.from_messages([
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}? Also, What is your name?"),
    ]
)

messages = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(messages)

# PromptTemplate: 문자열 템플릿에 변수 자리를 두고, format 메서드로 해당 변수에 값을 대입해 완성된 프롬프트를 만들어준다.
# ChatPromptTemplate: 메시지 리스트 형태로 시스템, AI, 사용자 메시지 등을 템플릿화한 뒤, format_messages 메서드로 변수 치환을 통해 최종 메시지 시퀀스를 생성한다.
# from_template 메서드: 템플릿 문자열로부터 PromptTemplate나 ChatPromptTemplate 인스턴스를 생성한다.
# format/format_messages 메서드: 템플릿 안의 변수 자리(placeholder)에 실제 값을 주입하여 최종적으로 모델에게 보낼 프롬프트나 메시지 리스트를 만든다.

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",") # strip 메서드는 문자열 양 끝의 공백을 제거, split 메서드는 문자열을 구분자로 나누어 리스트로 반환
        return list(map(str.strip, items)) # map 함수를 통해 리스트의 각 요소에 strip 메서드를 적용

p = CommaOutputParser()

p.parse("Hello ,how ,are ,you")

In [ ]:
template = ChatPromptTemplate.from_messages([
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. DO NOT reply with anything else."),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(max_items=10, question="what are the colors?")

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

# template = ChatPromptTemplate.from_messages: 시스템과 사용자 메시지를 템플릿화한 뒤, format_messages 메서드로 변수 치환을 통해 최종 메시지 시퀀스를 생성한다.
# prompt = template.format_messages: 사용자 메시지에 대한 템플릿을 만들고, 이를 모델에 전달하여 응답을 받는다.
# parse 메서드: 모델의 응답을 파싱하여 원하는 형태로 변환한다.


In [ ]:
chain = template | chat | CommaOutputParser()

chain.invoke({ "max_items": 3, "question": "what are the pokemons?" })

# chain = template | chat | CommaOutputParser(): 템플릿, 챗 모델, 파서를 연결하여 하나의 파이프라인을 만든다.
# 체인의 invoke 메서드를 호출하면, 체인의 각 구성 요소가 순차적으로 실행되어 최종 결과를 반환한다.
# 체인은 다음과 같은 경우에 사용하기 좋다.
# 	1.	여러 단계로 이루어진 작업을 하나의 단일 작업으로 묶어야 할 때, eg. chain = template | chat | CommaOutputParser()
# 	2.	여러 단계의 작업을 반복적으로 수행해야 할 때, eg. for data in dataset: chain.invoke(data)
# 	3.	여러 단계의 작업을 조합하여 새로운 작업을 만들어야 할 때, eg. chain1 = template1 | chat1 | CommaOutputParser(); chain2 = template2 | chat2 | CommaOutputParser()
# 	4.	여러 단계의 작업을 병렬로 실행하고, 결과를 합쳐야 할 때, eg. chain1 = template1 | chat1 | CommaOutputParser(); chain2 = template2 | chat2 | CommaOutputParser(); chain = chain1 & chain2
# 	5.	여러 단계의 작업을 조건부로 실행해야 할 때, eg. chain1 = template1 | chat1 | CommaOutputParser(); chain2 = template2 | chat2 | CommaOutputParser(); chain = chain1 | chain2

In [ ]:
from langchain.callbacks import StreamingStdOutCallbackHandler
# StreamingStdOutCallbackHandler: 콜백 핸들러의 기본 클래스로, 콜백을 통해 모델의 출력을 스트리밍하거나 저장할 수 있다.
# langchain.callbacks는 LangChain 라이브러리의 콜백 기능을 제공하는 모듈이다.  모듈을 사용하면 LangChain의 다양한 이벤트에 대해 사용자 정의 동작을 정의할 수 있는데, 예를 들어 특정 이벤트가 발생할 때 로그를 남기거나, 추가적인 처리를 수행할 수 있다.

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]);
# streaming=True: 모델의 출력을 스트리밍하도록 설정

chef_template = ChatPromptTemplate.from_messages([
        ("system", "You are a world-class chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
        ("human", "I want to cook {cuisine} food"),
    ]
)

chef_chain = chef_template | chat
# chef_chain: 템플릿과 챗 모델을 연결하여 체인을 만든다.

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
        ("human", "{recipe}")
    ]
)

veg_chain = veg_chef_prompt | chat
# veg_chain: 템플릿과 챗 모델을 연결하여 체인을 만든다.

final_chain = { "recipe": chef_chain } | veg_chain
# final_chain: 두 체인을 조건부로 실행하는 체인을 만든다.
# { "recipe": chef_chain }: 조건식을 통해 chef_chain을 실행한다. recipe 변수에 chef_chain의 결과를 전달한다.

final_chain.invoke({ "cuisine": "Indian" })
# final_chain.invoke: 조건부 체인을 실행한다. cuisine 변수에 "Indian" 값을 주입하여 실행한다

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

# PromptTemplate: 문자열 템플릿에 변수 자리를 두고, format 메서드로 해당 변수에 값을 대입해 완성된 프롬프트를 만들어준다.
# FewShotPromptTemplate: 예시(샷) 목록과 템플릿을 결합하여, 주어진 예시와 함께 프롬프트를 생성한다. 예시를 통해 모델의 응답 품질을 향상시킨다.
# 차이점: PromptTemplate은 단순히 변수 치환을 통해 프롬프트를 생성하는 반면, FewShotPromptTemplate은 예시를 포함하여 더 풍부한 문맥을 제공한다.
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# chat.predict("What do you know about France?")
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
# example_prompt: 프롬프트의 예시 템플릿입니다. 이 변수는 미리 정의된 프롬프트 형식을 포함하고 있습니다.
# examples: 프롬프트에 사용할 예시 데이터입니다. 이 변수는 여러 개의 예시를 포함하는 리스트일 가능성이 큽니다.
# suffix: 프롬프트의 끝에 추가될 문자열입니다. 여기서는 "Human: What do you know about {country}?"라는 문자열이 사용됩니다. {country}는 input_variables에 의해 동적으로 대체될 변수입니다.
# input_variables: 프롬프트에서 동적으로 대체될 변수들의 리스트입니다. 여기서는 ["country"]가 사용됩니다.
# 이 코드는 주어진 예시와 템플릿을 기반으로 특정 국가에 대한 정보를 묻는 프롬프트를 생성하는 데 사용됩니다. 예를 들어, country 변수에 "France"가 들어가면 최종 프롬프트는 "Human: What do you know about France?"가 됩니다.

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
        Here is what I know:
        Capital: Berlin
        Language: German
        Food: Sausages and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Berlin\n        Language: German\n        Food: Sausages and Sauerkraut\n        Currency: Euro')